# GeoPandas

### Description

>The goal of GeoPandas is to make working with geospatial data in python easier. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS.

***
First we set up the tools we will be using in this notebook.
1. `pandas` for reading in the CSV file containing information about 2,797 4-year institution across the United States.
2. `geopandas` for working with geospatial data.
3. `geoplot` to more easily create nice-looking maps with geopandas.
4. `pathlib.Path` so that the filepaths work regardless of your operating system.
5. `tools.tree`, you may notice there's a `tools.py` script in our notebooks folder. It includes a `tree` function that helps us display the contents of a directory.

In [1]:
import pandas as pd
import geopandas as gpd
import geoplot as gplt
from pathlib import Path
from tools import tree

%matplotlib inline

We include the `magic` 
```python
%matplotlib inline
```
to display our `matplotlib` plots in our jupyter notebook.

In [2]:
DATA_DIRECTORY = Path("../data/")

In [3]:
tree(DATA_DIRECTORY)

In our `data/external/` folder you'll find the data as it comes from [IPEDS](https://nces.ed.gov/ipeds/) (Integrated Postsecondary Education Data System) from the National Center for Education Statistics (NCES).

In our `data/processed/` folder you'll find the data already processed / cleaned. That's the file we'll be using.

In [4]:
colleges = pd.read_csv(DATA_DIRECTORY / 'processed' / 'colleges_in_US.csv')

In [5]:
colleges.head()

In [6]:
colleges.shape

You'll notice we have `Longitude` and `Latitude` columns in our dataset. We can use those to create a `geometry` column. The added value of GeoPandas is the ability to manipulate geometries like you would other data types in pandas.

Once you have a `geometry` column in `pandas` your `DataFrame` becomes a... `GeoDataFrame`

In [7]:
geo_colleges = gpd.GeoDataFrame(colleges, geometry = gpd.points_from_xy(colleges['Longitude'], colleges['Latitude']))

***
`Pandas` uses `matplotlib` to plot data. You can use the `.plot()` method on a dataframe and it'll create the default chart for each of the columns

In [8]:
colleges.plot()

`GeoPandas` default is to plot the geometry column.

In [9]:
geo_colleges.plot();

You can specify many parameters to make your plot nicer

In [10]:
geo_colleges.plot(markersize = 4, color = 'red', figsize = (8,8))

***
Enter, `geoplot`.

`geoplot` is a high-level Python geospatial plotting library. It’s an extension to `cartopy` and `matplotlib` which makes mapping easy: like `seaborn` for geospatial. It comes with the following features:

    High-level plotting API: geoplot is cartographic plotting for the 90% of use cases. All of the standard-bearermaps that you’ve probably seen in your geography textbook are easily accessible.

    Native projection support: The most fundamental peculiarity of geospatial plotting is projection: how do you unroll a sphere onto a flat surface (a map) in an accurate way? The answer depends on what you’re trying to depict. geoplot provides these options.

    Compatibility with matplotlib: While matplotlib is not a good fit for working with geospatial data directly, it’s a format that’s well-incorporated by other tools.


`geoplot` defaults are nicer-looking that `geopandas` defaults

In [11]:
gplt.pointplot(geo_colleges);

and it's easy to change the projection to display your data

In [12]:
import geoplot.crs as gcrs

In [13]:
gplt.pointplot(geo_colleges)

In [14]:
gplt.pointplot(geo_colleges, projection = gcrs.AlbersEqualArea())

In [15]:
gplt.pointplot(geo_colleges, projection = gcrs.LambertCylindrical())

And even includes _webmaps_

In [16]:
ax = gplt.webmap(geo_colleges, projection=gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax=ax)

***
Since `GeoPandas` is essentially geospatially-enabled `pandas` you can manipulate the dataframe like you would any other dataset.

In [17]:
mask_lower48 = (geo_colleges['State abbreviation'] == 'HI') | (geo_colleges['State abbreviation'] == 'AK')

In [18]:
geo_colleges[mask_lower48].head()

In [19]:
geo_colleges[~mask_lower48].head()

In [20]:
geo_colleges = geo_colleges[~mask_lower48].copy()

In [21]:
ax = gplt.webmap(geo_colleges, projection = gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax = ax, )

And because it's `matplotlib` under the hood, it's highly customizable.

In [22]:
VAR_OF_INTEREST = 'Undergraduate enrollment'

In [23]:
ax = gplt.webmap(geo_colleges, projection = gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax = ax, hue = VAR_OF_INTEREST, k = 8,);

In [24]:
ax = gplt.webmap(geo_colleges, projection = gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax = ax, hue = VAR_OF_INTEREST, k = 8, legend = True);

In [25]:
ax = gplt.webmap(geo_colleges, projection = gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax = ax, hue = VAR_OF_INTEREST, k = 8, legend = True, cmap = 'Blues');

In [26]:
ax = gplt.webmap(geo_colleges, projection = gcrs.WebMercator())
gplt.pointplot(geo_colleges, ax = ax, hue = VAR_OF_INTEREST, k = 5, legend = True, cmap = 'Blues', scale = VAR_OF_INTEREST, legend_var = 'scale', limits = (1, 10));

***
GeoPandas has many useful tools for analysis. 

For example, let's say we wanted to know if there are any areas in the U.S. where there are no 4-year higher education institutions nearby. There, we would be looking at areas and we'll need `polygons` and right now we have only `points` for each of our institutions.

First, step is to see what reference system our data is in.

In [27]:
geo_colleges.crs

Since we created the geometries from a CSV, there's no explicit CRS attatched to our `GeoDataFrame`

In [28]:
geo_colleges.head(2)

But we can see it's using degrees as its unit and if you're familiar with the data you may be able to infer what the CRS is.

In [29]:
geo_colleges.crs = "+init=epsg:4326"

Also, you can check `GeoPandas`' documentation http://geopandas.org/projections.html

Changing a projection in GeoPandas is as simple as using `.to_crs()`

In [30]:
geo_colleges_meters = geo_colleges.to_crs(epsg = 3857)
geo_colleges_meters.head()

Let's look at 4-year institutions in CA.

In [31]:
mask_CA = geo_colleges_meters['State abbreviation'] == 'CA'
ca_colleges = geo_colleges_meters[mask_CA].copy()

In [33]:
ca_colleges.head()

To be able to map that on our webmaps we'll change the CRS back to 4326

In [57]:
ca_colleges = ca_colleges.to_crs(epsg = 4326)

In [58]:
ax = gplt.webmap(ca_colleges, projection = gcrs.WebMercator())
gplt.pointplot(ca_colleges, ax = ax, hue = VAR_OF_INTEREST, k = 5, legend = True, cmap = 'Blues', scale = VAR_OF_INTEREST, legend_var = 'scale', limits = (5,20), legend_kwargs={'frameon': False, 'loc': 'upper right'}, legend_values=[0, 10000, 20000, 30000, 40000],
    legend_labels=['0 undergrads', '10,000', '20,000', '30,000', '40,000'],
);

One way to look at it it's through a Kernel Density Estimation plot (or KDE plot)

In [60]:
ax = gplt.webmap(ca_colleges, zorder=1, projection= gcrs.WebMercator())
gplt.kdeplot(ca_colleges,  cmap='Reds', shade=True, ax = ax, alpha = 0.6)

A _more accurate_ way to do this would be to create actual X-mile areas around each of the institutions. 

This is how you'd do that in `GeoPandas`

First, we transform our GeoDataFrame to epsg 3857 which uses meters as its unit of measure, create a 50-mile buffer and transform that back to epsg 4326 so we can map that using our webmaps.

In [66]:
areas = ca_colleges.to_crs(epsg = 3857)['geometry'].buffer(50 * 1609).to_crs(epsg = 4326)

In [67]:
areas.head()

In [68]:
ax = gplt.webmap(areas, figsize=(12,12), projection = gcrs.WebMercator(), zorder=1)
gplt.polyplot(areas, ax = ax, edgecolor = 'black', zorder = 2)

In [79]:
areas = gpd.GeoDataFrame(areas)

areas.columns = ['geometry']

areas.head()

In [82]:
areas['50-mile radius'] = 'Yes'

In [83]:
areas.head()

In [85]:
areas_agg = areas.dissolve(by = '50-mile radius')

In [86]:
ax = gplt.webmap(areas_agg, figsize=(12,12), projection = gcrs.WebMercator(), zorder=1)
gplt.polyplot(areas_agg, ax = ax, edgecolor = 'black', zorder = 2)

In [ ]:
contiguous_usa = gpd.read_file(gplt.datasets.get_path("contiguous_usa"))

In [96]:
ca = contiguous_usa[contiguous_usa['state'] == 'California']

In [97]:
ca

In [98]:
clipped_areas = gpd.overlay(areas_agg, ca, how = 'intersection')

In [99]:
ax = gplt.webmap(clipped_areas, figsize=(12,12), projection = gcrs.WebMercator(), zorder=1)
gplt.polyplot(clipped_areas, ax = ax, edgecolor = 'black', zorder = 2, facecolor = 'red', alpha = 0.4)

In [102]:
missing_areas = gpd.overlay(clipped_areas, ca, how = "symmetric_difference")

In [103]:
ax = gplt.webmap(missing_areas, figsize=(12,12), projection = gcrs.WebMercator(), zorder=1)
gplt.polyplot(missing_areas, ax = ax, edgecolor = 'black', zorder = 2, facecolor = 'red', alpha = 0.4)

***
# Rest of US?

In [104]:
us_areas = geo_colleges.to_crs(epsg = 3857)['geometry'].buffer(50 * 1609).to_crs(epsg = 4326)

In [107]:
us_areas = gpd.GeoDataFrame(us_areas)

us_areas.columns = ['geometry']

us_areas['50-mile radius'] = 'Yes'

us_areas.head()

In [114]:
us_areas_agg = us_areas.dissolve(by = '50-mile radius')
us_areas.crs = {"init": "epsg:4326"}

In [115]:
clipped_us_areas = gpd.overlay(us_areas_agg, contiguous_usa, how = 'intersection')

In [116]:
missing_us_areas = gpd.overlay(clipped_us_areas, contiguous_usa, how = "symmetric_difference")

In [122]:
ax = gplt.webmap(missing_us_areas, figsize=(12,12), projection = gcrs.WebMercator(), zorder=1)
gplt.polyplot(missing_us_areas, ax = ax, edgecolor = 'black', zorder = 2,  )